In [ ]:
import pandas as pd
from glob import glob
import numpy as np

In [ ]:
all_results = glob('../max_probe_fine_tune_seeds/results/*/*.csv')
all_baseline_paths = glob('../baseline_eval_seeds/results/*/*.csv')
len(all_results), len(all_baseline_paths)

In [ ]:
our_method = 'AdUE (OUR)'

columns_name_mapping = {
    'w/o l2 norm | md_relative_roc_auc' : 'RMD',
    'l2 norm | md_relative_roc_auc' : 'RMD l2 norm',

    'w/o l2 norm | md_roc_auc' : 'MD',
    'l2 norm | md_roc_auc' : 'MD l2 norm',

    'w/o l2 norm | rde_n_components_256_roc_auc' : 'RDE',
    'l2 norm | rde_n_components_256_roc_auc' : 'RDE l2 norm',
    
    'l2 norm | test-base-max-prob-roc-auc': 'SR',
    
    'ablation_bce' : 'Loss: BCE',
    'ablation_bce_l2sp' : 'Loss: BCE+L2SP',
    'ablation_bce_reg' : 'Loss: BCE+reg',
    'all_reg' : 'Loss: BCE+reg+L2SP',
    'ablation_cls_random' : 'Full loss + rand cls',
    'ablation_linear_random' : 'Full loss + random linear',
    'full' : 'AdUE (OUR)',
    'test-fine-tune-max-prob-roc-auc' : 'AdUE (OUR)',
}


In [ ]:
baseline_columns = [
    'w/o l2 norm | md_relative_roc_auc', 
    'l2 norm | md_relative_roc_auc',

    'w/o l2 norm | md_roc_auc',
    'l2 norm | md_roc_auc',

    'w/o l2 norm | rde_n_components_256_roc_auc',
    'l2 norm | rde_n_components_256_roc_auc',
    
    'l2 norm | test-base-max-prob-roc-auc'
]

In [ ]:
model_name_mapping = {
    'Qwen/Qwen2.5-7B' : 'Qwen',
    'google/electra-base-discriminator' : 'Electra',
    'meta-llama/Llama-2-7b-hf' : 'LLama',
    'roberta-base' : 'Roberta'
}

dataset_name_mapping = {
    '20newsgroups' : 'News',
    'cola' : 'Cola',
    'SST2' : 'SST2',
    'SST5' : 'SST5',
    'toxigen' : 'Toxigen',
}

In [ ]:
all_dfs = []
for path in all_results:
    all_dfs.append(pd.read_csv(path))
all_dfs = pd.concat(all_dfs).reset_index(drop=True)
all_dfs = all_dfs.rename(columns_name_mapping, axis='columns')
all_dfs['model'] = all_dfs['model'].map(model_name_mapping)
all_dfs['dataset'] = all_dfs['dataset'].map(dataset_name_mapping)
all_dfs = all_dfs.rename(columns_name_mapping, axis='columns')


In [ ]:
baseline_columns = ['md_marginal_roc_auc', 'md_roc_auc', 'md_relative_roc_auc', 'rde_n_components_256_roc_auc', 'test-base-max-prob-roc-auc']

In [ ]:
all_baselines = []
for path in all_baseline_paths:
    all_baselines.append(pd.read_csv(path))
all_baselines = pd.concat(all_baselines).reset_index(drop=True)


all_baselines = all_baselines.pivot(
    index=['model', 'dataset', 'seed', 'train_on_dataset'], 
    columns='normalization', 
    values=baseline_columns
).rename({
    True : 'l2 norm',
    False : 'w/o l2 norm',
}, axis='columns')

all_baselines = all_baselines.swaplevel(0,1, axis='columns')
all_baselines = all_baselines.sort_index(axis=1)

all_baselines.columns = all_baselines.columns.map(' | '.join).str.strip('|')
all_baselines = all_baselines.reset_index().rename(columns_name_mapping, axis='columns')
all_baselines['model'] = all_baselines['model'].map(model_name_mapping)
all_baselines['dataset'] = all_baselines['dataset'].map(dataset_name_mapping)

In [ ]:
all_dfs = all_dfs[all_dfs['train_on_dataset'] == 'train']
all_baselines = all_baselines[all_baselines['train_on_dataset'] == 'train']

In [ ]:
columns_to_join_results = ['model', 'dataset', 'seed']
columns_to_gb = ['model', 'dataset']

In [ ]:
assert len(all_results) == 700
assert len(all_baseline_paths) == 200

In [ ]:
t = all_dfs.groupby(['model', 'dataset', 'grid'])[['adapter']].count()
assert len(t[t['adapter'] != 5]) == 0

In [ ]:
all_dfs = all_dfs.pivot(index=columns_to_join_results, columns='grid', values=our_method).reset_index().rename(columns_name_mapping, axis='columns')

In [ ]:
all_methods = [
    'RMD',
    'RMD l2 norm',
    'MD',
     'MD l2 norm',
     'RDE',
     'RDE l2 norm',
     'SR',
     'AdUE (OUR)',
     'Loss: BCE',
     'A: BCE+L2SP',
     'A: BCE+reg',
     'A: BCE+reg+L2SP',
     'A: rand cls',
     'A: random linear'
]

main_table_methods = [
    # 'RMD',
    'RMD l2 norm',
    # 'MD',
    'MD l2 norm',
    # 'RDE',
    'RDE l2 norm',
    'SR',
    'AdUE (OUR)',
]

ablation_methods = [
    'AdUE (OUR)',
    'Loss: BCE',
    'Loss: BCE+L2SP',
    'Loss: BCE+reg',
    # 'Loss: BCE+reg+L2SP',
    'Full loss + rand cls',
    'Full loss + random linear',
]

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
result_with_baselines = all_baselines.set_index(columns_to_join_results).join(all_dfs.set_index(columns_to_join_results))

In [ ]:
def modify_cells_with_mask(df, mask, modification_func, inplace=False):
    """
    Modify DataFrame cells where mask is True using a modification function.
    
    Args:
        df: pandas DataFrame
        mask: Boolean mask (same shape as df) indicating cells to modify
        modification_func: Function to apply to each selected cell
        inplace: Whether to modify the original DataFrame
        
    Returns:
        Modified DataFrame (if inplace=False) or None (if inplace=True)
    """
    if not inplace:
        df = df.copy()
    
    # Apply modification only where mask is True
    for col in df.columns:
        for idx in df.index:
            if mask.loc[idx, col]:
                df.at[idx, col] = modification_func(df.at[idx, col])
    
    return None if inplace else df

In [ ]:
def get_mean_std_table(df, methods):
    df = df.loc[:, methods]
    mean_values = df.groupby(columns_to_gb).agg('mean').T
    std_values = df.groupby(columns_to_gb).agg('std').T

    rank_table = mean_values.rank(axis='rows', ascending=False)
    cols = pd.MultiIndex.from_product([['Rank'],['Rank']])
    mean_rank = pd.DataFrame(rank_table.mean(axis=1), columns=cols)
    mean_rank_str = mean_rank.astype(str)
    mean_rank_rank = mean_rank.rank(axis='rows', ascending=True)

    mean_values_str = (mean_values * 100).round(1).astype(str)
    std_values_str = (std_values * 100).round(1).astype(str)

    str_df = mean_values_str + '±' + std_values_str
    str_df = modify_cells_with_mask(str_df, rank_table==1, lambda x: f'\\textbf{{{x}}}')
    str_df = modify_cells_with_mask(str_df, rank_table==2, lambda x: f'\\underline{{{x}}}')
    
    mean_rank_str = modify_cells_with_mask(mean_rank_str, mean_rank_rank==1, lambda x: f'\\textbf{{{x}}}')
    mean_rank_str = modify_cells_with_mask(mean_rank_str, mean_rank_rank==2, lambda x: f'\\underline{{{x}}}')
    f = pd.concat((str_df, mean_rank_str), axis='columns')
    
    return f.loc[methods].to_latex(escape=False, column_format='c' * (len(f.columns) + 1)).replace('AdUE (OUR)', '\\textbf{{AdUE (OUR)}}')

In [ ]:
print(get_mean_std_table(result_with_baselines.loc[['Roberta', 'Electra']], main_table_methods))

In [ ]:
print(get_mean_std_table(result_with_baselines.loc[['LLama', 'Qwen']], main_table_methods))

In [ ]:
print(get_mean_std_table(result_with_baselines.loc[['Roberta', 'Electra']], ablation_methods))

In [ ]:
print(get_mean_std_table(result_with_baselines.loc[['LLama', 'Qwen']], ablation_methods))

In [ ]:
# def get_rank_mean(df, methods):
#     df = df.loc[:, methods]
#     mean_values = df.groupby(columns_to_gb).agg('mean').T
#     std_values = df.groupby(columns_to_gb).agg('std').T

#     rank_table = mean_values.rank(axis='rows', ascending=False)
#     cols = pd.MultiIndex.from_product([['Rank'],['Rank']])
#     mean_rank = pd.DataFrame(rank_table.mean(axis=1), columns=cols)
#     mean_rank_str = mean_rank.astype(str)
#     mean_rank_rank = mean_rank.rank(axis='rows', ascending=True)

#     mean_values_str = (mean_values * 100).round(1).astype(str)
#     std_values_str = (std_values * 100).round(1).astype(str)

#     str_df = mean_values_str + '±' + std_values_str
#     str_df = modify_cells_with_mask(str_df, rank_table==1, lambda x: f'\\textbf{{{x}}}')
#     str_df = modify_cells_with_mask(str_df, rank_table==2, lambda x: f'\\underline{{{x}}}')
    
#     mean_rank_str = modify_cells_with_mask(mean_rank_str, mean_rank_rank==1, lambda x: f'\\textbf{{{x}}}')
#     mean_rank_str = modify_cells_with_mask(mean_rank_str, mean_rank_rank==2, lambda x: f'\\underline{{{x}}}')
    
#     f = pd.concat((str_df, mean_rank_str), axis='columns')
    
#     return f.loc[methods].to_latex(escape=False, column_format='c' * (len(f.columns) + 1)).replace('AdUE (OUR)', '\\textbf{{AdUE (OUR)}}')